In [ ]:
import pydicom # for reading dicom files
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import cv2
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
 



# patients paths
patients = []
for i in range(15):
    s1 = "3Dircadb1/3Dircadb1."
    s2 = "/PATIENT_DICOM"
    patients.append(s1+str(i+1)+s2)


In [ ]:
#load dicom patients
patients_slices =[]
for patient in patients:
#     labels_df.get_value(patient, 'cancer')
    path = patient
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [pydicom.read_file(path+"/" + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    patients_slices.append(slices)


In [ ]:
#getting Hounsfield units (HU)
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

patients_pixels = []
for patients in patients_slices:
    first_patient = patients
    first_patient_pixels = get_pixels_hu(first_patient)
    patients_pixels.append(first_patient_pixels)




In [ ]:
#normalize patients to focus on liver
MIN_BOUND = - 200
MAX_BOUND = 500

normalized = []
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image


for pat in patients_pixels[1]:
    normalized.append(normalize(patients_pixels[i])) 



In [ ]:
#resamble voxel to 1mm, 1mm, 1mm
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    #spacing = np.array( shape=([scan[0].SliceThickness] , scan[0].PixelSpacing, scan[0].PixelSpacing), dtype=np.float32)
    spacing = np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    image = image.atype(np.float32)image = image.astype(np.float32)
    return image, new_spacing

patients_resampled = []
for i in range(1):
    pix_resampled, spacing = resample( normalized[i], liver_slices, [1,1,1])
    patients_resampled.append(pix_resampled)

In [ ]:
np.save('offline-pattients.npy',patients_resampled)